Get original DataFrame

In [1]:
import sys
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
sys.path.insert(0, os.path.abspath('../..'))

from python_scripts.api_calls import fetch_item_from_api, fetch_item_to_df, fetch_items, get_cookie_from_blob
from make_graphs import plot_historic_price, plot_historic_volume,  plot_lognormalised_historic_price, plot_lognormalised_historic_volume, plot_simple_moving_average, plot_exponential_moving_average, plot_weighted_least_squares, plot_bollinger_bands, plot_relative_strength_index, plot_money_flow_index
from python_scripts.calculate_metrics import create_ln_df, calculate_sma, calculate_ema, calculate_bollinger_bands, calculate_price_percentage_change, calculate_relative_strength_index, calculate_market_cap, calculate_money_flow_index, calculate_market_cap_jupyter

dailyCookie = get_cookie_from_blob()
items = fetch_items()

current_item = fetch_item_to_df(items[4], dailyCookie)
#print(items[4])

#print(current_item.tail())
#print(non_aggregated_item.tail())

df = current_item

In [3]:
processed_df = df["price_usd"]
tf.random.set_seed(0)

#processed_df['y'] = processed_df[0].shift(-1)

dataset = processed_df.values
dataset = dataset.astype('float32')
print(dataset)
print(processed_df)
# normalize the dataset
#scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)


[6.433 3.563 2.707 ... 6.379 6.379 6.318]
date
2016-11-29    6.4330
2016-11-30    3.5630
2016-12-01    2.7070
2016-12-02    2.0750
2016-12-03    1.6100
               ...  
2024-08-05    6.3195
2024-08-06    6.4130
2024-08-07    6.3790
2024-08-08    6.3790
2024-08-09    6.3180
Freq: D, Name: price_usd, Length: 2811, dtype: float64


In [4]:
#split into training and testing
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size

train, test = dataset[0:train_size], dataset[train_size:len(dataset)]
